In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [3]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/230126_ppk/'
sp_plasmids_file = folder + 'plasmids_sp.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex-pl.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex_mt.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'230126_plasmids_sp_updated.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 6.875, '1_P1': 0.15, '1_P2': 0.525, '1_P3': 0.15, '1_P4': 0.675, '1_P5': 0.15, '1_P8': 0.15, '1_P9': 0.15, '1_P17': 0.1, '1_P18': 0.075}


In [4]:
protocol_genex_df = protocol_genex_df[0]

In [5]:
vol_table_df

,Label,water,biotin,Pyc,Fdh,P70_GFP,Mdh,Mdh_10x,ME2,TdcB,PLP,SHMT,ICL,MS,ACS,Chi6,ppk12,Mdh_2x,ppk_Mt,ppk_Pa
0,ppk_Mt,0.3,0.05,0.175,0.05,0.0,0.05,NaN,NaN,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025,0.0
1,ppk_Pa,0.3,0.05,0.175,0.05,0.0,0.05,NaN,NaN,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.025
2,ppk_neg,0.325,0.05,0.175,0.05,0.0,0.05,NaN,NaN,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,mt_only,0.675,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.075,0.0
4,pa_only,0.7,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.05
5,gfp_1,0.75,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
6,gfp_2,0.725,0.0,0.0,0.0,0.025,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
7,gfp_3,0.725,0.0,0.0,0.0,0.025,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
8,gfp_4,0.675,0.0,0.0,0.0,0.075,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9,gfp_5,0.575,0.0,0.0,0.0,0.175,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [6]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'P24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

std
ppk_Mt
ppk_Pa
ppk_neg
gfp_1
gfp_2
gfp_3
gfp_4
mt_only
gfp_5
pa_only
gfp_6
TXTL needed: 27.0


In [7]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [8]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [9]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'buffers-mt.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 9.0, '1_B1': 9.8, '1_C1': 4.5, '1_D1': 9.0, '1_E1': 40.0, '1_E2': 5.0, '1_F1': 0.45, '1_G1': 7.2, '1_H1': 18.0, '1_J1': 9.2, '1_K1': 0.4, '1_L1': 0.4, '1_M1': 18.75, '1_N1': 9.0, '1_O1': 9.0, '1_P1': 18.4, '2_A2': 391.9, '3_G7': 10.0, '3_H7': 15.0, '3_I7': 15.0, '3_J7': 15.0, '3_O6': 7.5, '3_P6': 7.5}


In [11]:
sps[0]

,Label,Well,Concentration,Volume
Item,,,,
Biotin,Biotin,A1,50.00000,60
ATP,ATP_1x,B1,50.00000,60
acetyl-CoA,acetyl-CoA,C1,100.00000,60
Mg-Act,Mg-Act,"D1,D2",100.00000,"60,60"
HCT,HCT,"E1,E2,E3",100.00000,"60,60.0,60.0"
Kan,Kan,F1,1000.00000,60
formate,formate,"G1,G2",932.95676,"60,60"
HCO3,HCO3,"H1,H2",250.00000,"60,60"
KOH,KOH,"I1,I2",1000.00000,"60,60"


In [12]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)